WEB SCRAPING

In [1]:
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from PIL import Image
from io import BytesIO

In [6]:
def setup_driver():
    """Configura el driver de Selenium con Chrome."""
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Ejecutar sin abrir ventana (modo headless)
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    return webdriver.Chrome(options=options)

def scroll_down(driver):
    """Hace scroll en la página para cargar más imágenes."""
    body = driver.find_element(By.TAG_NAME, "body")
    for _ in range(75):  # Aumentamos el número de scrolls
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(2)

def get_image_urls(driver, search_query, num_images):
    """Obtiene las URLs de imágenes de Google Images."""
    search_url = f"https://www.google.com/search?q={search_query}&tbm=isch"
    driver.get(search_url)
    time.sleep(3)  # Esperar más tiempo a que la página cargue completamente
    scroll_down(driver)
    
    img_elements = driver.find_elements(By.CSS_SELECTOR, "img")  # Buscar todas las imágenes
    img_urls = []
    
    print(f"Se encontraron {len(img_elements)} imágenes en la página.")
    
    for img in img_elements:
        try:
            img_url = img.get_attribute("src") or img.get_attribute("data-src")  # Extraer src o data-src
            if img_url and img_url.startswith("http") and len(img_urls) < num_images:
                img_urls.append(img_url)
                print(f"URL obtenida: {img_url}")
        except Exception as e:
            print(f"Error obteniendo URL de imagen: {e}")
    
    print(f"Se obtuvieron {len(img_urls)} URLs de imágenes.")
    return img_urls

def download_images(image_urls, folder):
    """Descarga y guarda las imágenes en la carpeta especificada."""
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    for i, url in enumerate(image_urls):
        try:
            response = requests.get(url, timeout=10)
            image = Image.open(BytesIO(response.content))
            
            # Convertir imágenes con modos incompatibles a RGB
            if image.mode in ("P", "RGBA"):
                image = image.convert("RGB")
            
            image_path = os.path.join(folder, f"image_{i+1}.jpg")
            image.save(image_path, "JPEG")
            print(f"Imagen guardada: {image_path}")
        except Exception as e:
            print(f"Error descargando la imagen {i+1}: {e}")

def main():
    driver = setup_driver()
    
    categories = {"apples": "apple fruit", "bananas": "banana fruit"}  # Búsqueda optimizada
    num_images = 800  # Número de imágenes por categoría
    
    for category, query in categories.items():
        print(f"Descargando imágenes de: {category}")
        image_urls = get_image_urls(driver, query, num_images)
        download_images(image_urls, folder=f"images/{category}")
    
    driver.quit()
    print("Descarga completada")

if __name__ == "__main__":
    main()


Descargando imágenes de: apples
Se encontraron 1697 imágenes en la página.
URL obtenida: https://www.google.com/logos/doodles/2025/rise-of-the-half-moon-march-6753651837110631-shs.png
URL obtenida: https://www.google.com/logos/doodles/2025/rise-of-the-half-moon-march-6753651837110631-shs.png
URL obtenida: https://www.google.com/logos/doodles/2025/rise-of-the-half-moon-march-6753651837110631-s.png
URL obtenida: https://www.google.com/logos/doodles/2025/rise-of-the-half-moon-march-6753651837110631-lsg.png
URL obtenida: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR4fAMaPU_WoLF-RbJlR4HpDRQuSF_cK4PzYG6RBFPXVNLAeoOBNPqC2RB4&s
URL obtenida: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSvL8nN6Vdtfjgttd1AnSjY3rSCAMpM7r3ZJK6hGFik14hf93A1MnfH-e5D&s
URL obtenida: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSq1jVI7dyXEU9yRqYg6Q3YBgWLIIKOcR5Mai1Upce6OFOnf_4QV07j7ihG&s
URL obtenida: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS3rek1XQfwD2o0et8T33v0mn57hxtTYxYMXp

c:\Users\colom\OneDrive - ITESO\iteso\6to semestre\deep_learning\venv\Lib\site-packages\PIL\Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Imagen guardada: images/apples\image_2.jpg
Imagen guardada: images/apples\image_3.jpg
Imagen guardada: images/apples\image_4.jpg
Imagen guardada: images/apples\image_5.jpg
Imagen guardada: images/apples\image_6.jpg
Imagen guardada: images/apples\image_7.jpg
Imagen guardada: images/apples\image_8.jpg
Imagen guardada: images/apples\image_9.jpg
Error descargando la imagen 10: cannot identify image file <_io.BytesIO object at 0x0000018F7C544130>
Imagen guardada: images/apples\image_11.jpg
Imagen guardada: images/apples\image_12.jpg
Imagen guardada: images/apples\image_13.jpg
Imagen guardada: images/apples\image_14.jpg
Imagen guardada: images/apples\image_15.jpg
Imagen guardada: images/apples\image_16.jpg
Imagen guardada: images/apples\image_17.jpg
Imagen guardada: images/apples\image_18.jpg
Imagen guardada: images/apples\image_19.jpg
Imagen guardada: images/apples\image_20.jpg
Imagen guardada: images/apples\image_21.jpg
Imagen guardada: images/apples\image_22.jpg
Imagen guardada: images/ap

FILTRAR IMAGENES CON RESNET

In [3]:
import os
import shutil
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

# Cargar modelo pre-entrenado
model = ResNet50(weights='imagenet')

def classify_image(img_path):
    """Clasifica una imagen con ResNet50 y devuelve las predicciones top-3."""
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    
    preds = model.predict(img_array)
    decoded_preds = decode_predictions(preds, top=3)[0]  # Obtener las 3 predicciones más probables
    return decoded_preds

def filter_images(input_folder, output_folder, valid_labels):
    """Filtra imágenes y mueve solo las que sean manzanas o bananas."""
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for img_name in os.listdir(input_folder):
        img_path = os.path.join(input_folder, img_name)
        try:
            predictions = classify_image(img_path)
            top_labels = [pred[1] for pred in predictions]  # Extraer nombres de las predicciones
            print(f"Imagen: {img_name}, Predicciones: {top_labels}")
            
            if any(label in valid_labels for label in top_labels):  # Si alguna predicción coincide con las permitidas
                shutil.move(img_path, os.path.join(output_folder, img_name))
        except Exception as e:
            print(f"Error clasificando {img_name}: {e}")

def main():
    categories = {
        "apples": ["Granny_Smith", "red_apple", "green_apple", "apple"],
        "bananas": ["banana"]
    }  # Etiquetas de ImageNet para cada categoría
    
    input_base = "images/"
    output_base = "filtered_images/"
    
    for category, valid_labels in categories.items():
        print(f"Filtrando imágenes de {category}...")
        input_folder = os.path.join(input_base, category)
        output_folder = os.path.join(output_base, category)
        filter_images(input_folder, output_folder, valid_labels)
    
    print("Filtrado completado.")

if __name__ == "__main__":
    main()


Filtrando imágenes de apples...
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Imagen: image_1.jpg, Predicciones: ['mortarboard', 'chime', 'spotlight']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 285ms/step
Imagen: image_100.jpg, Predicciones: ['Granny_Smith', 'banana', 'fig']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step
Imagen: image_101.jpg, Predicciones: ['digital_clock', 'velvet', 'prayer_rug']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step
Imagen: image_102.jpg, Predicciones: ['earthstar', 'fig', 'hip']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step
Imagen: image_103.jpg, Predicciones: ['prayer_rug', 'pedestal', 'sliding_door']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step
Imagen: image_104.jpg, Predicciones: ['spaghetti_squash', 'pomegranate', 'banana']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step
Imagen: image_105.jpg, Predicciones: ['tray', 'digital_clock', 'studio_couch']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step
Imagen: image_106.jpg, Predicciones: ['pomegranate', 'wig', 'artichoke']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 310ms/step
Imagen: image_107